In [ ]:
import pandas as pd
import numpy as np
import os
import random
import seaborn as sns
import importlib
import yaml
import sys
import pygwalker as pyg
import streamlit as st
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import shutil

import pygwalker as pyg
import yaml
from io import StringIO


import functions as functions
importlib.reload(functions)

<module 'functions' from 'c:\\Users\\halewijnvanden\\OneDrive\\Documenten\\financials\\remake_buxfer\\streamlit_app\\functions.py'>

In [3]:
settings['transfer_between_own_account_categories']

NameError: name 'settings' is not defined

In [ ]:
import functions as functions
importlib.reload(functions)

settings = functions.load_settings('./dashboard_config.yaml')
temp_dir = './' + settings['temp_dir'] + '/'
data_repo_path = temp_dir + 'data/'
functions.make_directories([data_repo_path])

all_sheets = pd.read_excel('./data/finances_2025 (2).xlsx', sheet_name=None)
transaction_df = pd.concat(all_sheets.values(), ignore_index=True)
transaction_df['month_number']=transaction_df['Date'].dt.month
transaction_df['main_tag'] = transaction_df['tag'].apply(functions.get_main_category)
transaction_df['month'] = transaction_df['Date'].dt.to_period('M')
transaction_df.to_csv(data_repo_path + 'transaction_df.csv', index=False)


intra_account_transaction_filt = transaction_df['tag'].isin(settings['transfer_between_own_account_categories'])
refund_filt = transaction_df['tag'].isin(settings['refund_categories'])

core_transaction_df = transaction_df[~intra_account_transaction_filt & ~refund_filt]
core_transaction_df['inverted_amount'] = -core_transaction_df['amount'] 
core_transaction_df.to_csv(data_repo_path + 'core_transaction_df.csv', index=False)

monthly_category_df = core_transaction_df.groupby(['month', 'main_tag'])['amount'].sum().reset_index()
monthly_category_df['is_income_category'] = monthly_category_df['main_tag'].isin(settings['income_categories'])
monthly_category_df.to_csv(data_repo_path + 'monthly_category_df.csv', index=False)

spending_montly_category_df = monthly_category_df[monthly_category_df['is_income_category']==False].reset_index(drop = True)
spending_montly_category_df.to_csv(data_repo_path + 'spending_montly_category_df.csv', index=False)

montly_balance_df=monthly_category_df.groupby(['month', 'is_income_category'], as_index = False)['amount'].sum()
montly_balance_df.to_csv(data_repo_path + 'montly_balance_df.csv', index=False)

net_balance_df = montly_balance_df.groupby('month', as_index=False)['amount'].sum()
net_balance_df.to_csv(data_repo_path + 'net_balance_df.csv', index=False)

monthly_spend_by_category_summary_df = spending_montly_category_df.groupby(['month', 'main_tag'])['amount'].sum().reset_index()
total_spend_summary_df = spending_montly_category_df.groupby('month')['amount'].transform('sum')
monthly_spend_by_category_summary_df['percent'] = monthly_spend_by_category_summary_df['amount'] / total_spend_summary_df * 100
monthly_spend_by_category_summary_df['inverted_amount'] = -monthly_spend_by_category_summary_df['amount'] 
monthly_spend_by_category_summary_df.to_csv(data_repo_path + 'monthly_spend_by_category_summary_df.csv', index=False)

spend_by_category_percentage_df = monthly_spend_by_category_summary_df.groupby(['main_tag'], as_index = False)['percent'].mean()
spend_by_category_percentage_df.to_csv(data_repo_path + 'spend_by_category_percentage_df.csv', index=False)

refund_df = transaction_df[refund_filt]
refund_df.to_csv(data_repo_path + 'refund_df.csv', index=False)

C:\Users\halewijnvanden\AppData\Local\Temp\ipykernel_40880\3033306998.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import functions as functions
importlib.reload(functions)
settings = functions.load_settings('./dashboard_config.yaml')
functions.set_theme(settings)

In [12]:
os.listdir('.streamlit')

['config.toml']

In [ ]:
import functions as functions
importlib.reload(functions)


settings = functions.load_settings('./dashboard_config.yaml')

#--------------------- plot the balance figure -----------------------------

settings = functions.load_settings('./dashboard_config.yaml')['plots']['spending_income_plot']
functions.plot_income_spending(montly_balance_df,net_balance_df, settings)

In [ ]:
import functions as functions
importlib.reload(functions)
import pandas as pd
import plotly.graph_objects as go
settings = functions.load_settings('./dashboard_config.yaml')


functions.plot_monthly_spend_percentage(monthly_spend_by_category_summary_df,
                              settings['plots']['percentage_spend_by_category_plot'])

In [ ]:

avg_saving_per_month = int(pd.read_csv('./temp/data/net_balance_df.csv')['amount'].mean())

avg_saving_spending_df = pd.read_csv('./temp/data/montly_balance_df.csv').groupby(['is_income_category'],as_index = False)['amount'].mean()
avg_income_per_month = avg_saving_spending_df[avg_saving_spending_df['is_income_category'] == True].iloc[0]['amount']
avg_spending_per_month = avg_saving_spending_df[avg_saving_spending_df['is_income_category'] == False].iloc[0]['amount']

refund_summary_df = pd.read_csv('./temp/data/refund_df.csv').groupby(['main_tag'],as_index=False)['amount'].sum()
parental_refund = refund_summary_df[refund_summary_df['main_tag'] == 'parental_refund'].iloc[0]['amount']
pending_refund = refund_summary_df[refund_summary_df['main_tag'] == 'pending_refund'].iloc[0]['amount']
loan = refund_summary_df[refund_summary_df['main_tag'] == 'loan'].iloc[0]['amount']

In [58]:
monthly_spend_by_category_summary_df

,month,main_tag,amount,percent,inverted_amount
0,2025-01,entertainment,-60.49,5.468270,60.49
1,2025-01,food,-229.88,20.781052,229.88
2,2025-01,other,-126.57,11.441873,126.57
3,2025-01,rent,-228.28,20.636413,228.28
4,2025-01,sports,-143.96,13.013922,143.96
...,...,...,...,...,...
68,2025-09,gifts,-84.73,3.373506,84.73
69,2025-09,insurance,-149.50,5.952310,149.50
70,2025-09,other,-53.18,2.117350,53.18
71,2025-09,transportation,-231.98,9.236233,231.98


In [75]:
core_transaction_df[core_transaction_df['tag'].isna()]

,Date,card,description,amount,tag,main tag,inverted_amount,absolute_amount,proof,month_number,main_tag,month
172,2025-03-06,belfius_mastercard,MASTERCARD AFREKENING NUMMER 056,-9.99,NaN,NaN,9.99,NaN,NaN,3,unknown,2025-03


In [86]:
core_transaction_df.groupby(['tag'], as_index = False)['amount'].sum()

,tag,amount
0,entertainment,-218.41
1,entertainment/drinks,-381.43
2,food,-139.50
3,food/groceries,-1209.67
4,food/restaurants,-312.33
5,food/student_restaurants,-210.69
6,gifts,-129.63
7,income,22734.20
8,income/parental_income,3239.71
9,income/stocks_obligations_loans,345.80


In [ ]:
import functions as functions
importlib.reload(functions)

settings = functions.load_settings('./dashboard_config.yaml')
functions.treemap_df(core_transaction_df,
        settings['plots']['treemap_plot'])

In [ ]:
import functions as functions
importlib.reload(functions)
import plotly.express as px
df = px.data.tips()
fig = px.treemap(df, path=[px.Constant("all"), 'sex', 'day', 'time'],
                 values='total_bill', color='day')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [40]:
pd.read_csv('./temp/data/refund_df.csv')

,Date,card,description,amount,tag,main tag,inverted_amount,absolute_amount,proof,month_number,main_tag,month
0,2025-01-31,belfius_deposit,inschrijving mosbach colloqium,-150.00,pending_refund,NaN,NaN,NaN,NaN,1,pending_refund,2025-01
1,2025-01-27,belfius_deposit,slagerij t'kint,-24.35,parental_refund,NaN,NaN,NaN,NaN,1,parental_refund,2025-01
2,2025-01-27,belfius_deposit,Terugbetaling t kint 10 01 2025 Deel van de re...,27.40,parental_refund,NaN,NaN,NaN,NaN,1,parental_refund,2025-01
3,2025-01-20,belfius_deposit,slagerij t'kint,-19.75,parental_refund,NaN,NaN,NaN,NaN,1,parental_refund,2025-01
4,2025-01-20,belfius_deposit,Terugbetaling t kint,19.75,parental_refund,NaN,NaN,NaN,NaN,1,parental_refund,2025-01
5,2025-01-13,belfius_deposit,slagerij t'kint,-47.40,parental_refund,NaN,NaN,NaN,NaN,1,parental_refund,2025-01
6,2025-01-03,belfius_mastercard,inschrijving congres,-420.00,pending_refund,NaN,NaN,NaN,NaN,1,pending_refund,2025-01
7,2025-02-17,belfius_deposit,Terugbetaling t'Kint 24 Jan,24.35,parental_refund,NaN,NaN,NaN,NaN,2,parental_refund,2025-02
8,2025-02-03,belfius_mastercard,ik denk het treinticket voor naar Parijs te gaan,-86.00,pending_refund,NaN,NaN,NaN,NaN,2,pending_refund,2025-02
9,2025-02-03,belfius_mastercard,overnachting Parijs,-180.00,pending_refund,NaN,NaN,NaN,NaN,2,pending_refund,2025-02


In [31]:
avg_income_per_month

2924.412222222222